# Justin Zarkovacki 2/15/2023
# Transfer Learning MNIST -> KMNIST

# Prepare imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install optuna==3.0.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.2/147.2 KB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 KB 14.3 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=4c87b6ea827cbd2491bf3d1036553a0f784e5acdb74684d7749664a4bbea4a26
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
  A

# Transfer Learning Notebook

In [ ]:
import numpy as np
import os
import optuna
import random

import matplotlib
from matplotlib import pyplot as plt

import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.datasets import mnist
from keras.layers import Conv2D, Dropout, AveragePooling2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, Rescaling
from keras import Input, models, backend as K
from tensorflow.keras import layers, models

print("Done!")

Done!


# Function Definitions and Variables

In [ ]:
epochs = 12
img_rows, img_cols = 28, 28  # Image dimensions

def load(f):
    return np.load(f)['arr_0']
    
def initialize_data(train_im_file, test_im_file, train_lb_file, test_lb_file):
    train_images = load(train_im_file)
    test_images = load(test_im_file)
    train_labels = load(train_lb_file)
    test_labels = load(test_lb_file)
    
    if K.image_data_format() == 'channels_first':
        train_images = train_images.reshape(train_images.shape[0], 1, img_rows, img_cols)
        test_images = test_images.reshape(test_images.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        train_images = train_images.reshape(train_images.shape[0], img_rows, img_cols, 1)
        test_images = test_images.reshape(test_images.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)

    train_images = train_images.astype('float32')
    test_images = test_images.astype('float32')
    train_images /= 255
    test_images /= 255
    print('{} train samples, {} test samples'.format(len(train_images), len(test_images)))
    
    return tuple([train_images, test_images, train_labels, test_labels, input_shape])

# Helper to create the graphics
def create_visuals(graph_title, model_hist, test_images, test_labels):
    accuracy_data = model_hist.history['accuracy']
    val_accuracy_data = model_hist.history['val_accuracy']

    lower_bound = min(min(accuracy_data), min(val_accuracy_data))

    plt.plot(accuracy_data, label='Train Accuracy')
    plt.plot(val_accuracy_data, label = 'Validation Accuracy')

    plt.title(graph_title)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([lower_bound - 0.01, 1])
    plt.legend(loc='lower right')

    print(accuracy_data[-1])
    print(val_accuracy_data[-1])
#     test_loss, test_acc = model_hist.evaluate(test_images, test_labels, verbose=2)

print("Done!")

Done!


## Load Data

In [ ]:
prefix = '/content/drive/MyDrive/datasets//'
datasets = {
    "km_tr_i" : prefix + 'Kuzushiji-MNIST - train-imgs.npz',
    "km_te_i" : prefix + 'Kuzushiji-MNIST - test-imgs.npz',
    "km_tr_l" : prefix + 'Kuzushiji-MNIST - train-labels.npz',
    "km_te_l" : prefix + 'Kuzushiji-MNIST - test-labels.npz',
}

# Load MNIST Data
(mnist_train_images, mnist_train_labels), (mnist_test_images, mnist_test_labels) = mnist.load_data()
mnist_input_shape = (28, 28, 1)
mnist_classes = 10
mnist_epochs = 15

# Load KMNIST Data
dataset = initialize_data(datasets["km_tr_i"], datasets["km_te_i"], datasets["km_tr_l"], datasets["km_te_l"])
kmnist_train_images = dataset[0]
kmnist_test_images = dataset[1]
kmnist_train_labels = dataset[2]
kmnist_test_labels  = dataset[3]
kmnist_input_shape = dataset[4]
kmnist_classes = 10
kmnist_epochs = 15

batches = 128
num_trials = 25

print("Done!")

11490434/11490434 [==============================] - 0s 0us/step
60000 train samples, 10000 test samples
Done!


This notebook will create an ensemble model for KMNIST character recognition. It wil be composed of 2 basic models, and one transfer learning model. Knowledge from MNIST will be transfered to KMNIST.

# Creating KMNIST Model 1

In [ ]:
def kmnist_objective1(trial):
    # Define search space per trial (integer, categorical and floating point values)
    kern_size = trial.suggest_int('kernel_size', 2, 3)
    l1_filters = trial.suggest_int('first_layer_kernel', 32, 54)
    l2_filters = trial.suggest_int('second_layer_kernel', 20, 64)
    l1_activation = trial.suggest_categorical('first_layer_activation', ['relu', 'sigmoid', 'tanh'])
    l2_activation = trial.suggest_categorical('second_layer_activation', ['relu', 'sigmoid', 'tanh'])
    dropout = trial.suggest_float('dropout', 0.15, 0.3)
    average_pooling_size = trial.suggest_int('average_pooling_size', 2, 4)
    dense_layer_size = trial.suggest_int('dense_layer_size', 64, 80)
    dense_layer_activation = trial.suggest_categorical('dense_layer_activation', ['relu', 'sigmoid', 'tanh'])

    # Design model
    kmnist1_design = Sequential()
    kmnist1_design.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=kmnist_input_shape))
    kmnist1_design.add(Dropout(dropout))
    kmnist1_design.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=kmnist_input_shape))
    kmnist1_design.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
    kmnist1_design.add(Flatten())
    kmnist1_design.add(Dense(dense_layer_size, activation=dense_layer_activation))
    kmnist1_design.add(Dense(kmnist_classes))

    kmnist1_design.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

#     print(kmnist1_design.summary())

    callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

    kmnist1_history = kmnist1_design.fit(kmnist_train_images, kmnist_train_labels, epochs=kmnist_epochs, batch_size=batches,
                    callbacks=callback, validation_data=(kmnist_test_images, kmnist_test_labels))

    # Important metric for optuna to optimize over
    return kmnist1_history.history['val_accuracy'][-1]

In [ ]:
# Run Study 1
kmnist_study1 = optuna.create_study(direction='maximize', study_name="KMNIST-1")
kmnist_study1.optimize(kmnist_objective1, n_trials=num_trials)

In [ ]:
# Print the info from the best trial
print(f'Best trial info:\n{kmnist_study1.best_trial}\n')
for param, value in kmnist_study1.best_params.items():
    print(f'Param: {param}\tValue: {value}')

Best trial info:
FrozenTrial(number=11, values=[0.9496999979019165], datetime_start=datetime.datetime(2023, 2, 17, 19, 1, 50, 150095), datetime_complete=datetime.datetime(2023, 2, 17, 19, 3, 14, 414231), params={'kernel_size': 3, 'first_layer_kernel': 50, 'second_layer_kernel': 63, 'first_layer_activation': 'relu', 'second_layer_activation': 'relu', 'dropout': 0.1646000975336841, 'average_pooling_size': 4, 'dense_layer_size': 80, 'dense_layer_activation': 'relu'}, distributions={'kernel_size': IntDistribution(high=3, log=False, low=2, step=1), 'first_layer_kernel': IntDistribution(high=54, log=False, low=32, step=1), 'second_layer_kernel': IntDistribution(high=64, log=False, low=20, step=1), 'first_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'second_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'dropout': FloatDistribution(high=0.3, log=False, low=0.15, step=None), 'average_pooling_size': IntDistribution(high=4, lo

In [ ]:
# Optuna doesn't save the best model. You must rebuild it and save it.
kern_size	= 3
l1_filters = 50
l2_filters = 63
l1_activation = "relu"
l2_activation = "relu"
dropout	= 0.1646000975336841
average_pooling_size = 4
dense_layer_size	= 80
dense_layer_activation = "relu"

kmnist1 = Sequential()
kmnist1.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=kmnist_input_shape))
kmnist1.add(Dropout(dropout))
kmnist1.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=kmnist_input_shape))
kmnist1.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
kmnist1.add(Flatten())
kmnist1.add(Dense(dense_layer_size, activation=dense_layer_activation))
kmnist1.add(Dense(kmnist_classes))

kmnist1.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

# kmnist1.summary()

In [ ]:
tf.config.run_functions_eagerly(True)

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

kmnist1_optuna_history = kmnist1.fit(kmnist_train_images, kmnist_train_labels, epochs=kmnist_epochs, batch_size=batches,
                            callbacks=callback, validation_data=(kmnist_test_images, kmnist_test_labels))
kmnist1.save('/content/drive/MyDrive/saved_models/kmnist1.h5', save_format='h5')

Epoch 1/15


/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


469/469 [==============================] - 23s 48ms/step - loss: 0.4464 - sparse_categorical_crossentropy: 1.7126 - accuracy: 0.8651 - val_loss: 0.4669 - val_sparse_categorical_crossentropy: 1.6704 - val_accuracy: 0.8579
Epoch 2/15
469/469 [==============================] - 24s 52ms/step - loss: 0.1558 - sparse_categorical_crossentropy: 1.4541 - accuracy: 0.9534 - val_loss: 0.3582 - val_sparse_categorical_crossentropy: 1.5974 - val_accuracy: 0.8971
Epoch 3/15
469/469 [==============================] - 30s 63ms/step - loss: 0.1043 - sparse_categorical_crossentropy: 1.3610 - accuracy: 0.9693 - val_loss: 0.2885 - val_sparse_categorical_crossentropy: 1.5216 - val_accuracy: 0.9184
Epoch 4/15
469/469 [==============================] - 24s 52ms/step - loss: 0.0798 - sparse_categorical_crossentropy: 1.3201 - accuracy: 0.9761 - val_loss: 0.2722 - val_sparse_categorical_crossentropy: 1.4548 - val_accuracy: 0.9285
Epoch 5/15
469/469 [==============================] - 22s 48ms/step - loss: 0.0615 

# Creating KMNIST Model 2

In [ ]:
def kmnist_objective2(trial):
    # Define search space per trial (integer, categorical and floating point values)
    kern_size = trial.suggest_int('kernel_size', 2, 3)
    l1_filters = trial.suggest_int('first_layer_kernel', 20, 40)
    l2_filters = trial.suggest_int('second_layer_kernel', 40, 64)
    l1_activation = trial.suggest_categorical('first_layer_activation', ['relu', 'sigmoid', 'tanh'])
    l2_activation = trial.suggest_categorical('second_layer_activation', ['relu', 'sigmoid', 'tanh'])
    dropout = trial.suggest_float('dropout', 0.15, 0.3)
    average_pooling_size = trial.suggest_int('average_pooling_size', 2, 4)
    dense_layer_size = trial.suggest_int('dense_layer_size', 64, 80)
    dense_layer_activation = trial.suggest_categorical('dense_layer_activation', ['relu', 'sigmoid', 'tanh'])

    # Design model
    kmnist2_design = Sequential()
    kmnist2_design.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=kmnist_input_shape))
    kmnist2_design.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=kmnist_input_shape))
    kmnist2_design.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
    kmnist2_design.add(Dropout(dropout))
    kmnist2_design.add(Flatten())
    kmnist2_design.add(Dense(dense_layer_size, activation=dense_layer_activation))
    kmnist2_design.add(Dense(kmnist_classes))

    kmnist2_design.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

#     print(kmnist2_design.summary())

    callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

    kmnist2_history = kmnist2_design.fit(kmnist_train_images, kmnist_train_labels, epochs=kmnist_epochs, batch_size=batches,
                    callbacks=callback, validation_data=(kmnist_test_images, kmnist_test_labels))

    # Important metric for optuna to optimize over
    return kmnist2_history.history['val_accuracy'][-1]

In [ ]:
# Run Study 2
kmnist_study2 = optuna.create_study(direction='maximize', study_name="KMNIST-2")
kmnist_study2.optimize(kmnist_objective2, n_trials=30)

[I 2023-02-17 20:10:38,762] A new study created in memory with name: KMNIST-2
/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/15
469/469 [==============================] - 23s 49ms/step - loss: 0.4070 - sparse_categorical_crossentropy: 1.5540 - accuracy: 0.8791 - val_loss: 0.4054 - val_sparse_categorical_crossentropy: 1.4944 - val_accuracy: 0.8752
Epoch 2/15
469/469 [==============================] - 24s 51ms/step - loss: 0.1205 - sparse_categorical_crossentropy: 1.2513 - accuracy: 0.9644 - val_loss: 0.2932 - val_sparse_categorical_crossentropy: 1.3807 - val_accuracy: 0.9186
Epoch 3/15
469/469 [==============================] - 22s 47ms/step - loss: 0.0776 - sparse_categorical_crossentropy: 1.1624 - accuracy: 0.9768 - val_loss: 0.2650 - val_sparse_categorical_crossentropy: 1.3510 - val_accuracy: 0.9277
Epoch 4/15
469/469 [==============================] - 23s 49ms/step - loss: 0.0569 - sparse_categorical_crossentropy: 1.1292 - accuracy: 0.9830 - val_loss: 0.2664 - val_sparse_categorical_crossentropy: 1.3137 - val_accuracy: 0.9318
Epoch 5/15
469/469 [==============================] - 23s 49ms/step - lo

[I 2023-02-17 20:17:01,307] Trial 0 finished with value: 0.9466999769210815 and parameters: {'kernel_size': 3, 'first_layer_kernel': 24, 'second_layer_kernel': 56, 'first_layer_activation': 'relu', 'second_layer_activation': 'tanh', 'dropout': 0.192922226646244, 'average_pooling_size': 3, 'dense_layer_size': 72, 'dense_layer_activation': 'relu'}. Best is trial 0 with value: 0.9466999769210815.


Epoch 1/15
469/469 [==============================] - 23s 48ms/step - loss: 0.4501 - sparse_categorical_crossentropy: 1.5664 - accuracy: 0.8647 - val_loss: 0.4391 - val_sparse_categorical_crossentropy: 1.5004 - val_accuracy: 0.8639
Epoch 2/15
469/469 [==============================] - 23s 49ms/step - loss: 0.1714 - sparse_categorical_crossentropy: 1.2516 - accuracy: 0.9497 - val_loss: 0.3427 - val_sparse_categorical_crossentropy: 1.4123 - val_accuracy: 0.9046
Epoch 3/15
469/469 [==============================] - 22s 46ms/step - loss: 0.1235 - sparse_categorical_crossentropy: 1.1789 - accuracy: 0.9628 - val_loss: 0.2795 - val_sparse_categorical_crossentropy: 1.3232 - val_accuracy: 0.9192
Epoch 4/15
469/469 [==============================] - 23s 48ms/step - loss: 0.0949 - sparse_categorical_crossentropy: 1.1151 - accuracy: 0.9714 - val_loss: 0.2561 - val_sparse_categorical_crossentropy: 1.2694 - val_accuracy: 0.9268
Epoch 5/15
469/469 [==============================] - 24s 50ms/step - lo

[I 2023-02-17 20:23:23,790] Trial 1 finished with value: 0.9498000144958496 and parameters: {'kernel_size': 3, 'first_layer_kernel': 34, 'second_layer_kernel': 45, 'first_layer_activation': 'tanh', 'second_layer_activation': 'relu', 'dropout': 0.2623422535363248, 'average_pooling_size': 4, 'dense_layer_size': 80, 'dense_layer_activation': 'relu'}. Best is trial 1 with value: 0.9498000144958496.


...

[I 2023-02-17 22:54:44,497] Trial 27 finished with value: 0.9506999850273132 and parameters: {'kernel_size': 3, 'first_layer_kernel': 38, 'second_layer_kernel': 59, 'first_layer_activation': 'tanh', 'second_layer_activation': 'relu', 'dropout': 0.1731469164202171, 'average_pooling_size': 3, 'dense_layer_size': 78, 'dense_layer_activation': 'relu'}. Best is trial 22 with value: 0.9574000239372253.


Epoch 1/15
469/469 [==============================] - 23s 49ms/step - loss: 1.3412 - sparse_categorical_crossentropy: 2.3537 - accuracy: 0.5562 - val_loss: 1.0914 - val_sparse_categorical_crossentropy: 2.2561 - val_accuracy: 0.6599
Epoch 2/15
469/469 [==============================] - 22s 47ms/step - loss: 0.6086 - sparse_categorical_crossentropy: 1.5755 - accuracy: 0.8185 - val_loss: 0.8839 - val_sparse_categorical_crossentropy: 1.9563 - val_accuracy: 0.7171
Epoch 3/15
469/469 [==============================] - 23s 49ms/step - loss: 0.4840 - sparse_categorical_crossentropy: 1.4354 - accuracy: 0.8552 - val_loss: 0.7763 - val_sparse_categorical_crossentropy: 1.8239 - val_accuracy: 0.7528
Epoch 4/15
469/469 [==============================] - 23s 49ms/step - loss: 0.4140 - sparse_categorical_crossentropy: 1.3476 - accuracy: 0.8772 - val_loss: 0.6845 - val_sparse_categorical_crossentropy: 1.6902 - val_accuracy: 0.7852
Epoch 5/15
469/469 [==============================] - 22s 47ms/step - lo

[I 2023-02-17 23:00:26,480] Trial 28 finished with value: 0.8895999789237976 and parameters: {'kernel_size': 3, 'first_layer_kernel': 36, 'second_layer_kernel': 59, 'first_layer_activation': 'sigmoid', 'second_layer_activation': 'sigmoid', 'dropout': 0.27004965129708264, 'average_pooling_size': 4, 'dense_layer_size': 76, 'dense_layer_activation': 'sigmoid'}. Best is trial 22 with value: 0.9574000239372253.


Epoch 1/15
469/469 [==============================] - 24s 50ms/step - loss: 0.4842 - sparse_categorical_crossentropy: 1.6651 - accuracy: 0.8578 - val_loss: 0.6386 - val_sparse_categorical_crossentropy: 1.7680 - val_accuracy: 0.8046
Epoch 2/15
469/469 [==============================] - 21s 46ms/step - loss: 0.2563 - sparse_categorical_crossentropy: 1.4621 - accuracy: 0.9270 - val_loss: 0.4634 - val_sparse_categorical_crossentropy: 1.6021 - val_accuracy: 0.8642
Epoch 3/15
469/469 [==============================] - 23s 48ms/step - loss: 0.1745 - sparse_categorical_crossentropy: 1.3971 - accuracy: 0.9508 - val_loss: 0.3828 - val_sparse_categorical_crossentropy: 1.5440 - val_accuracy: 0.8883
Epoch 4/15
469/469 [==============================] - 23s 49ms/step - loss: 0.1357 - sparse_categorical_crossentropy: 1.3625 - accuracy: 0.9617 - val_loss: 0.3454 - val_sparse_categorical_crossentropy: 1.5233 - val_accuracy: 0.8987
Epoch 5/15
469/469 [==============================] - 22s 46ms/step - lo

[I 2023-02-17 23:06:48,827] Trial 29 finished with value: 0.9229000210762024 and parameters: {'kernel_size': 3, 'first_layer_kernel': 40, 'second_layer_kernel': 54, 'first_layer_activation': 'tanh', 'second_layer_activation': 'tanh', 'dropout': 0.19940119817010848, 'average_pooling_size': 3, 'dense_layer_size': 72, 'dense_layer_activation': 'tanh'}. Best is trial 22 with value: 0.9574000239372253.


In [ ]:
# Print the info from the best trial
print(f'Best trial info:\n{kmnist_study2.best_trial}\n')
for param, value in kmnist_study2.best_params.items():
    print(f'Param: {param}\tValue: {value}')

Best trial info:
FrozenTrial(number=22, values=[0.9574000239372253], datetime_start=datetime.datetime(2023, 2, 17, 22, 20, 7, 249500), datetime_complete=datetime.datetime(2023, 2, 17, 22, 25, 50, 261204), params={'kernel_size': 3, 'first_layer_kernel': 36, 'second_layer_kernel': 51, 'first_layer_activation': 'tanh', 'second_layer_activation': 'relu', 'dropout': 0.23565992578719316, 'average_pooling_size': 4, 'dense_layer_size': 80, 'dense_layer_activation': 'relu'}, distributions={'kernel_size': IntDistribution(high=3, log=False, low=2, step=1), 'first_layer_kernel': IntDistribution(high=40, log=False, low=20, step=1), 'second_layer_kernel': IntDistribution(high=64, log=False, low=40, step=1), 'first_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'second_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'dropout': FloatDistribution(high=0.3, log=False, low=0.15, step=None), 'average_pooling_size': IntDistribution(high=4, 

In [ ]:
# Optuna doesn't save the best model. You must rebuild it and save it.
kern_size	= 3
l1_filters = 36
l2_filters = 51
l1_activation = "tanh"
l2_activation = "relu"
dropout	= 0.23565992578719316
average_pooling_size = 4
dense_layer_size	= 80
dense_layer_activation = "relu"

kmnist2 = Sequential()
kmnist2.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=kmnist_input_shape))
kmnist2.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=kmnist_input_shape))
kmnist2.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
kmnist2.add(Dropout(dropout))
kmnist2.add(Flatten())
kmnist2.add(Dense(dense_layer_size, activation=dense_layer_activation))
kmnist2.add(Dense(kmnist_classes))

kmnist2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

# kmnist2.summary()

In [ ]:
tf.config.run_functions_eagerly(True)

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

kmnist2_optuna_history = kmnist2.fit(kmnist_train_images, kmnist_train_labels, epochs=kmnist_epochs, batch_size=batches,
                            callbacks=callback, validation_data=(kmnist_test_images, kmnist_test_labels))

kmnist2.save('/content/drive/MyDrive/saved_models/kmnist2.h5', save_format='h5')

Epoch 1/15


/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


469/469 [==============================] - 22s 46ms/step - loss: 0.4426 - sparse_categorical_crossentropy: 1.6114 - accuracy: 0.8669 - val_loss: 0.4276 - val_sparse_categorical_crossentropy: 1.5325 - val_accuracy: 0.8725
Epoch 2/15
469/469 [==============================] - 22s 48ms/step - loss: 0.1699 - sparse_categorical_crossentropy: 1.2471 - accuracy: 0.9488 - val_loss: 0.3394 - val_sparse_categorical_crossentropy: 1.3649 - val_accuracy: 0.8984
Epoch 3/15
469/469 [==============================] - 25s 53ms/step - loss: 0.1225 - sparse_categorical_crossentropy: 1.1881 - accuracy: 0.9626 - val_loss: 0.2780 - val_sparse_categorical_crossentropy: 1.2739 - val_accuracy: 0.9182
Epoch 4/15
469/469 [==============================] - 24s 51ms/step - loss: 0.0977 - sparse_categorical_crossentropy: 1.1112 - accuracy: 0.9708 - val_loss: 0.2547 - val_sparse_categorical_crossentropy: 1.2503 - val_accuracy: 0.9271
Epoch 5/15
469/469 [==============================] - 22s 47ms/step - loss: 0.0784 

# MNIST to KMNIST Transfer Learning

## Create MNIST Base

In [ ]:
def mnist_base_objective(trial):
    # Define search space per trial (integer, categorical and floating point values)
    kern_size = trial.suggest_int('kernel_size', 2, 3)
    l1_filters = trial.suggest_int('first_layer_kernel', 20, 40)
    l2_filters = trial.suggest_int('second_layer_kernel', 40, 64)
    l1_activation = trial.suggest_categorical('first_layer_activation', ['relu', 'sigmoid', 'tanh'])
    l2_activation = trial.suggest_categorical('second_layer_activation', ['relu', 'sigmoid', 'tanh'])
    dropout = trial.suggest_float('dropout', 0.15, 0.3)
    average_pooling_size = trial.suggest_int('average_pooling_size', 2, 4)
    dense_layer_size = trial.suggest_int('dense_layer_size', 64, 80)
    dense_layer_activation = trial.suggest_categorical('dense_layer_activation', ['relu', 'sigmoid', 'tanh'])

    # Design model
    mnist_base_design = Sequential()
    mnist_base_design.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=mnist_input_shape))
    mnist_base_design.add(Dropout(dropout))
    mnist_base_design.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=mnist_input_shape))
    # mnist_base_design.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
    mnist_base_design.add(Flatten())
    mnist_base_design.add(Dense(dense_layer_size, activation=dense_layer_activation))
    mnist_base_design.add(Dense(mnist_classes))

    mnist_base_design.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

#     print(mnist_base_design.summary())

    callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

    mnist_base_history = mnist_base_design.fit(mnist_train_images, mnist_train_labels,
                    epochs=mnist_epochs, batch_size=batches, callbacks=callback,
                    validation_data=(mnist_test_images, mnist_test_labels))

    # Important metric for optuna to optimize over
    return mnist_base_history.history['val_accuracy'][-1]

In [ ]:
# Run Study 3
mnist_base_study = optuna.create_study(direction='maximize', study_name="MNIST-Base")
mnist_base_study.optimize(mnist_base_objective, n_trials=10)

[I 2023-02-19 04:40:57,539] A new study created in memory with name: MNIST-Base


Epoch 1/15
469/469 [==============================] - 8s 11ms/step - loss: 2.3263 - sparse_categorical_crossentropy: 2.9315 - accuracy: 0.1067 - val_loss: 2.3033 - val_sparse_categorical_crossentropy: 2.3026 - val_accuracy: 0.1135
Epoch 2/15
469/469 [==============================] - 5s 10ms/step - loss: 2.3032 - sparse_categorical_crossentropy: 2.3026 - accuracy: 0.1070 - val_loss: 2.3025 - val_sparse_categorical_crossentropy: 2.3026 - val_accuracy: 0.1135
Epoch 3/15
469/469 [==============================] - 5s 10ms/step - loss: 2.3029 - sparse_categorical_crossentropy: 2.3026 - accuracy: 0.1089 - val_loss: 2.3022 - val_sparse_categorical_crossentropy: 2.3026 - val_accuracy: 0.1010
Epoch 4/15
469/469 [==============================] - 5s 10ms/step - loss: 2.3029 - sparse_categorical_crossentropy: 2.3026 - accuracy: 0.1077 - val_loss: 2.3017 - val_sparse_categorical_crossentropy: 2.3026 - val_accuracy: 0.1135
Epoch 5/15
469/469 [==============================] - 5s 11ms/step - loss: 2

[I 2023-02-19 04:41:44,365] Trial 0 finished with value: 0.10279999673366547 and parameters: {'kernel_size': 3, 'first_layer_kernel': 40, 'second_layer_kernel': 48, 'first_layer_activation': 'tanh', 'second_layer_activation': 'sigmoid', 'dropout': 0.1842557295414608, 'average_pooling_size': 2, 'dense_layer_size': 72, 'dense_layer_activation': 'sigmoid'}. Best is trial 0 with value: 0.10279999673366547.


Epoch 1/15
469/469 [==============================] - 6s 9ms/step - loss: 0.2171 - sparse_categorical_crossentropy: 1.7050 - accuracy: 0.9388 - val_loss: 0.1104 - val_sparse_categorical_crossentropy: 1.6502 - val_accuracy: 0.9700
Epoch 2/15
469/469 [==============================] - 4s 9ms/step - loss: 0.0835 - sparse_categorical_crossentropy: 1.7257 - accuracy: 0.9761 - val_loss: 0.0905 - val_sparse_categorical_crossentropy: 1.7406 - val_accuracy: 0.9769
Epoch 3/15
469/469 [==============================] - 4s 8ms/step - loss: 0.0572 - sparse_categorical_crossentropy: 1.7723 - accuracy: 0.9835 - val_loss: 0.0908 - val_sparse_categorical_crossentropy: 1.8115 - val_accuracy: 0.9777
Epoch 4/15
469/469 [==============================] - 4s 9ms/step - loss: 0.0422 - sparse_categorical_crossentropy: 1.8016 - accuracy: 0.9869 - val_loss: 0.0789 - val_sparse_categorical_crossentropy: 1.8013 - val_accuracy: 0.9806
Epoch 5/15
469/469 [==============================] - 4s 9ms/step - loss: 0.0372

[I 2023-02-19 04:43:07,286] Trial 1 finished with value: 0.9772999882698059 and parameters: {'kernel_size': 3, 'first_layer_kernel': 32, 'second_layer_kernel': 47, 'first_layer_activation': 'relu', 'second_layer_activation': 'tanh', 'dropout': 0.2598515559909706, 'average_pooling_size': 2, 'dense_layer_size': 72, 'dense_layer_activation': 'relu'}. Best is trial 1 with value: 0.9772999882698059.


...

[I 2023-02-19 04:49:44,778] Trial 7 finished with value: 0.9825999736785889 and parameters: {'kernel_size': 2, 'first_layer_kernel': 29, 'second_layer_kernel': 52, 'first_layer_activation': 'relu', 'second_layer_activation': 'tanh', 'dropout': 0.17435848899058518, 'average_pooling_size': 4, 'dense_layer_size': 65, 'dense_layer_activation': 'sigmoid'}. Best is trial 7 with value: 0.9825999736785889.


Epoch 1/15
469/469 [==============================] - 7s 12ms/step - loss: 0.2577 - sparse_categorical_crossentropy: 1.4429 - accuracy: 0.9310 - val_loss: 0.0957 - val_sparse_categorical_crossentropy: 1.4146 - val_accuracy: 0.9739
Epoch 2/15
469/469 [==============================] - 5s 10ms/step - loss: 0.0744 - sparse_categorical_crossentropy: 1.3684 - accuracy: 0.9769 - val_loss: 0.0963 - val_sparse_categorical_crossentropy: 1.4067 - val_accuracy: 0.9780
Epoch 3/15
469/469 [==============================] - 5s 11ms/step - loss: 0.0487 - sparse_categorical_crossentropy: 1.3591 - accuracy: 0.9856 - val_loss: 0.0891 - val_sparse_categorical_crossentropy: 1.3960 - val_accuracy: 0.9790
Epoch 4/15
469/469 [==============================] - 5s 10ms/step - loss: 0.0374 - sparse_categorical_crossentropy: 1.3915 - accuracy: 0.9885 - val_loss: 0.0993 - val_sparse_categorical_crossentropy: 1.4194 - val_accuracy: 0.9790
Epoch 5/15
469/469 [==============================] - 4s 9ms/step - loss: 0.

[I 2023-02-19 04:50:57,482] Trial 8 finished with value: 0.9818999767303467 and parameters: {'kernel_size': 2, 'first_layer_kernel': 32, 'second_layer_kernel': 61, 'first_layer_activation': 'relu', 'second_layer_activation': 'tanh', 'dropout': 0.23127498120973827, 'average_pooling_size': 2, 'dense_layer_size': 73, 'dense_layer_activation': 'relu'}. Best is trial 7 with value: 0.9825999736785889.


Epoch 1/15
469/469 [==============================] - 6s 9ms/step - loss: 2.3442 - sparse_categorical_crossentropy: 2.7161 - accuracy: 0.1050 - val_loss: 2.3032 - val_sparse_categorical_crossentropy: 2.3062 - val_accuracy: 0.1135
Epoch 2/15
469/469 [==============================] - 4s 8ms/step - loss: 2.3027 - sparse_categorical_crossentropy: 2.3052 - accuracy: 0.1099 - val_loss: 2.3026 - val_sparse_categorical_crossentropy: 2.3056 - val_accuracy: 0.1028
Epoch 3/15
469/469 [==============================] - 4s 9ms/step - loss: 2.3030 - sparse_categorical_crossentropy: 2.3060 - accuracy: 0.1087 - val_loss: 2.3019 - val_sparse_categorical_crossentropy: 2.3048 - val_accuracy: 0.1135
Epoch 4/15
469/469 [==============================] - 4s 8ms/step - loss: 2.3031 - sparse_categorical_crossentropy: 2.3063 - accuracy: 0.1076 - val_loss: 2.3015 - val_sparse_categorical_crossentropy: 2.3024 - val_accuracy: 0.1135
Epoch 5/15
469/469 [==============================] - 4s 8ms/step - loss: 2.3029

[I 2023-02-19 04:51:18,812] Trial 9 finished with value: 0.11349999904632568 and parameters: {'kernel_size': 2, 'first_layer_kernel': 22, 'second_layer_kernel': 44, 'first_layer_activation': 'tanh', 'second_layer_activation': 'sigmoid', 'dropout': 0.22928162915639017, 'average_pooling_size': 3, 'dense_layer_size': 77, 'dense_layer_activation': 'sigmoid'}. Best is trial 7 with value: 0.9825999736785889.


In [ ]:
# Print the info from the best trial
print(f'Best trial info:\n{mnist_base_study.best_trial}\n')
for param, value in mnist_base_study.best_params.items():
    print(f'Param: {param}\tValue: {value}')

Best trial info:
FrozenTrial(number=7, values=[0.9825999736785889], datetime_start=datetime.datetime(2023, 2, 19, 4, 48, 39, 409443), datetime_complete=datetime.datetime(2023, 2, 19, 4, 49, 44, 777393), params={'kernel_size': 2, 'first_layer_kernel': 29, 'second_layer_kernel': 52, 'first_layer_activation': 'relu', 'second_layer_activation': 'tanh', 'dropout': 0.17435848899058518, 'average_pooling_size': 4, 'dense_layer_size': 65, 'dense_layer_activation': 'sigmoid'}, distributions={'kernel_size': IntDistribution(high=3, log=False, low=2, step=1), 'first_layer_kernel': IntDistribution(high=40, log=False, low=20, step=1), 'second_layer_kernel': IntDistribution(high=64, log=False, low=40, step=1), 'first_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'second_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'dropout': FloatDistribution(high=0.3, log=False, low=0.15, step=None), 'average_pooling_size': IntDistribution(high=4,

In [ ]:
# Optuna doesn't save the best model. You must rebuild it and save it.
kern_size	= 2
l1_filters = 29
l2_filters = 52
l1_activation = "relu"
l2_activation = "tanh"
dropout	= 0.17435848899058518
average_pooling_size = 4
dense_layer_size	= 65
dense_layer_activation = "sigmoid"

mnist_base = Sequential()
mnist_base.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=mnist_input_shape))
mnist_base.add(Dropout(dropout))
mnist_base.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=mnist_input_shape))
mnist_base.add(Flatten())
mnist_base.add(Dense(dense_layer_size, activation=dense_layer_activation))
mnist_base.add(Dense(mnist_classes))

mnist_base.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

mnist_optuna_history = mnist_base.fit(mnist_train_images, mnist_train_labels, batch_size=batches, epochs=epochs,
                                callbacks=callback, validation_data=(mnist_test_images, mnist_test_labels))

mnist_base.save('/content/drive/MyDrive/saved_models/mnist_base.h5', save_format='h5')

Epoch 1/12


/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


469/469 [==============================] - 22s 47ms/step - loss: 0.2552 - sparse_categorical_crossentropy: 0.9303 - accuracy: 0.9429 - val_loss: 0.0936 - val_sparse_categorical_crossentropy: 0.6725 - val_accuracy: 0.9782
Epoch 2/12
469/469 [==============================] - 21s 45ms/step - loss: 0.0775 - sparse_categorical_crossentropy: 0.6272 - accuracy: 0.9817 - val_loss: 0.0687 - val_sparse_categorical_crossentropy: 0.6201 - val_accuracy: 0.9825
Epoch 3/12
469/469 [==============================] - 23s 48ms/step - loss: 0.0539 - sparse_categorical_crossentropy: 0.5720 - accuracy: 0.9865 - val_loss: 0.0645 - val_sparse_categorical_crossentropy: 0.5701 - val_accuracy: 0.9803
Epoch 4/12
469/469 [==============================] - 23s 48ms/step - loss: 0.0424 - sparse_categorical_crossentropy: 0.5619 - accuracy: 0.9893 - val_loss: 0.0633 - val_sparse_categorical_crossentropy: 0.5910 - val_accuracy: 0.9803
Epoch 5/12
469/469 [==============================] - 21s 44ms/step - loss: 0.0360 

## Create KMNIST Top

In [ ]:
import os
import random
from skimage.transform import rescale
from skimage import io

# Loading a base model requires you pop all dimensionality flattening layers
def load_base_model(filename):
    bm = models.load_model(filename)
    bm.pop()
    bm.pop()
    bm.pop()
    bm.trainable = False
    return bm

In [ ]:
def kmnist_top_objective(trial):
    # Define search space per trial (integer, categorical and floating point values)
    kern_size = trial.suggest_int('kernel_size', 2, 3)
    l1_filters = trial.suggest_int('first_layer_kernel', 32, 54)
    l2_filters = trial.suggest_int('second_layer_kernel', 64, 96)
    l1_activation = trial.suggest_categorical('first_layer_activation', ['relu', 'sigmoid', 'tanh'])
    l2_activation = trial.suggest_categorical('second_layer_activation', ['relu', 'sigmoid', 'tanh'])
    dropout = trial.suggest_float('dropout', 0.15, 0.3)
    average_pooling_size = trial.suggest_int('average_pooling_size', 2, 4)
    dense_layer_size = trial.suggest_int('dense_layer_size', 64, 128)
    dense_layer_activation = trial.suggest_categorical('dense_layer_activation', ['relu', 'sigmoid', 'tanh'])

    base_model = load_base_model('/content/drive/MyDrive/saved_models/mnist_base.h5')
    kmnist_in_shape = (28, 28, 1)

    # Design model
    kmnist_top_design = Sequential()
    kmnist_top_design.add(base_model)
    kmnist_top_design.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=kmnist_in_shape))
    kmnist_top_design.add(Dropout(dropout))
    kmnist_top_design.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=kmnist_in_shape))
    kmnist_top_design.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
    kmnist_top_design.add(Flatten())
    kmnist_top_design.add(Dense(dense_layer_size, activation=dense_layer_activation))
    kmnist_top_design.add(Dense(kmnist_classes))

    kmnist_top_design.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

#     print(kmnist_top_design.summary())

    callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

    kmnist_top_history = kmnist_top_design.fit(kmnist_train_images, kmnist_train_labels, epochs=kmnist_epochs, batch_size=batches,
                    callbacks=callback, validation_data=(kmnist_test_images, kmnist_test_labels))

    # Important metric for optuna to optimize over
    return kmnist_top_history.history['val_accuracy'][-1]

In [ ]:
# Run Study 4
kmnist_TL_study = optuna.create_study(direction='maximize', study_name="KMNIST-TL-Results")
kmnist_TL_study.optimize(kmnist_top_objective, n_trials=num_trials)

[I 2023-02-19 05:21:04,071] A new study created in memory with name: KMNIST-TL-Results


Epoch 1/15


/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


469/469 [==============================] - 25s 52ms/step - loss: 2.1060 - sparse_categorical_crossentropy: 7.0936 - accuracy: 0.2058 - val_loss: 1.4344 - val_sparse_categorical_crossentropy: 2.7679 - val_accuracy: 0.5556
Epoch 2/15
469/469 [==============================] - 24s 51ms/step - loss: 0.5199 - sparse_categorical_crossentropy: 1.6127 - accuracy: 0.8400 - val_loss: 0.6330 - val_sparse_categorical_crossentropy: 1.8243 - val_accuracy: 0.8112
Epoch 3/15
469/469 [==============================] - 26s 55ms/step - loss: 0.2297 - sparse_categorical_crossentropy: 1.3578 - accuracy: 0.9328 - val_loss: 0.4557 - val_sparse_categorical_crossentropy: 1.5845 - val_accuracy: 0.8651
Epoch 4/15
469/469 [==============================] - 25s 52ms/step - loss: 0.1610 - sparse_categorical_crossentropy: 1.2886 - accuracy: 0.9538 - val_loss: 0.4038 - val_sparse_categorical_crossentropy: 1.4992 - val_accuracy: 0.8840
Epoch 5/15
469/469 [==============================] - 25s 53ms/step - loss: 0.1212 

[I 2023-02-19 05:27:26,560] Trial 0 finished with value: 0.9308000206947327 and parameters: {'kernel_size': 2, 'first_layer_kernel': 42, 'second_layer_kernel': 81, 'first_layer_activation': 'sigmoid', 'second_layer_activation': 'relu', 'dropout': 0.23745963299235384, 'average_pooling_size': 3, 'dense_layer_size': 116, 'dense_layer_activation': 'tanh'}. Best is trial 0 with value: 0.9308000206947327.


...

[I 2023-02-19 07:45:38,107] Trial 22 finished with value: 0.9524999856948853 and parameters: {'kernel_size': 3, 'first_layer_kernel': 34, 'second_layer_kernel': 91, 'first_layer_activation': 'relu', 'second_layer_activation': 'relu', 'dropout': 0.16690854650458012, 'average_pooling_size': 3, 'dense_layer_size': 107, 'dense_layer_activation': 'relu'}. Best is trial 22 with value: 0.9524999856948853.


Epoch 1/15
469/469 [==============================] - 25s 52ms/step - loss: 0.3990 - sparse_categorical_crossentropy: 1.4711 - accuracy: 0.8794 - val_loss: 0.4149 - val_sparse_categorical_crossentropy: 1.4005 - val_accuracy: 0.8819
Epoch 2/15
469/469 [==============================] - 24s 52ms/step - loss: 0.1271 - sparse_categorical_crossentropy: 1.0178 - accuracy: 0.9623 - val_loss: 0.2824 - val_sparse_categorical_crossentropy: 1.1887 - val_accuracy: 0.9185
Epoch 3/15
469/469 [==============================] - 25s 53ms/step - loss: 0.0808 - sparse_categorical_crossentropy: 0.9399 - accuracy: 0.9754 - val_loss: 0.2320 - val_sparse_categorical_crossentropy: 1.0804 - val_accuracy: 0.9359
Epoch 4/15
469/469 [==============================] - 25s 53ms/step - loss: 0.0567 - sparse_categorical_crossentropy: 0.8839 - accuracy: 0.9828 - val_loss: 0.2196 - val_sparse_categorical_crossentropy: 1.1021 - val_accuracy: 0.9419
Epoch 5/15
469/469 [==============================] - 26s 56ms/step - lo

[I 2023-02-19 07:52:00,789] Trial 23 finished with value: 0.9519000053405762 and parameters: {'kernel_size': 3, 'first_layer_kernel': 38, 'second_layer_kernel': 92, 'first_layer_activation': 'relu', 'second_layer_activation': 'relu', 'dropout': 0.1513735513263988, 'average_pooling_size': 3, 'dense_layer_size': 108, 'dense_layer_activation': 'relu'}. Best is trial 22 with value: 0.9524999856948853.


Epoch 1/15
469/469 [==============================] - 27s 56ms/step - loss: 0.3734 - sparse_categorical_crossentropy: 1.6124 - accuracy: 0.8862 - val_loss: 0.3883 - val_sparse_categorical_crossentropy: 1.5342 - val_accuracy: 0.8882
Epoch 2/15
469/469 [==============================] - 25s 53ms/step - loss: 0.1137 - sparse_categorical_crossentropy: 1.2678 - accuracy: 0.9668 - val_loss: 0.2486 - val_sparse_categorical_crossentropy: 1.3695 - val_accuracy: 0.9274
Epoch 3/15
469/469 [==============================] - 25s 54ms/step - loss: 0.0701 - sparse_categorical_crossentropy: 1.1438 - accuracy: 0.9793 - val_loss: 0.2382 - val_sparse_categorical_crossentropy: 1.2821 - val_accuracy: 0.9346
Epoch 4/15
469/469 [==============================] - 25s 54ms/step - loss: 0.0507 - sparse_categorical_crossentropy: 1.0940 - accuracy: 0.9845 - val_loss: 0.2070 - val_sparse_categorical_crossentropy: 1.2234 - val_accuracy: 0.9451
Epoch 5/15
469/469 [==============================] - 25s 52ms/step - lo

[I 2023-02-19 07:58:23,289] Trial 24 finished with value: 0.9527999758720398 and parameters: {'kernel_size': 3, 'first_layer_kernel': 39, 'second_layer_kernel': 96, 'first_layer_activation': 'relu', 'second_layer_activation': 'relu', 'dropout': 0.15487543805908655, 'average_pooling_size': 3, 'dense_layer_size': 122, 'dense_layer_activation': 'relu'}. Best is trial 24 with value: 0.9527999758720398.


In [ ]:
# Print the info from the best trial
print(f'Best trial info:\n{kmnist_TL_study.best_trial}\n')
for param, value in kmnist_TL_study.best_params.items():
    print(f'Param: {param}\tValue: {value}')

Best trial info:
FrozenTrial(number=24, values=[0.9527999758720398], datetime_start=datetime.datetime(2023, 2, 19, 7, 52, 0, 790831), datetime_complete=datetime.datetime(2023, 2, 19, 7, 58, 23, 288938), params={'kernel_size': 3, 'first_layer_kernel': 39, 'second_layer_kernel': 96, 'first_layer_activation': 'relu', 'second_layer_activation': 'relu', 'dropout': 0.15487543805908655, 'average_pooling_size': 3, 'dense_layer_size': 122, 'dense_layer_activation': 'relu'}, distributions={'kernel_size': IntDistribution(high=3, log=False, low=2, step=1), 'first_layer_kernel': IntDistribution(high=54, log=False, low=32, step=1), 'second_layer_kernel': IntDistribution(high=96, log=False, low=64, step=1), 'first_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'second_layer_activation': CategoricalDistribution(choices=('relu', 'sigmoid', 'tanh')), 'dropout': FloatDistribution(high=0.3, log=False, low=0.15, step=None), 'average_pooling_size': IntDistribution(high=4, l

In [ ]:
# Optuna doesn't save the best model. You must rebuild it and save it.
base_model = load_base_model('/content/drive/MyDrive/saved_models/mnist_base.h5')
kern_size	= 3
l1_filters = 39
l2_filters = 96
l1_activation = "relu"
l2_activation = "relu"
dropout	= 0.15487543805908655
average_pooling_size = 3
dense_layer_size	= 122
dense_layer_activation = "relu"

kmnist_TL = Sequential()
kmnist_TL.add(base_model)
kmnist_TL.add(Conv2D(l1_filters, kernel_size=kern_size, activation=l1_activation, input_shape=kmnist_input_shape))
kmnist_TL.add(Dropout(dropout))
kmnist_TL.add(Conv2D(l2_filters, kernel_size=kern_size, activation=l2_activation, input_shape=kmnist_input_shape))
kmnist_TL.add(AveragePooling2D((average_pooling_size, average_pooling_size)))
kmnist_TL.add(Flatten())
kmnist_TL.add(Dense(dense_layer_size, activation=dense_layer_activation))
kmnist_TL.add(Dense(kmnist_classes))

kmnist_TL.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

kmnist_TL_optuna_history = kmnist_TL.fit(kmnist_train_images, kmnist_train_labels, epochs=kmnist_epochs, batch_size=batches,
                    callbacks=callback, validation_data=(kmnist_test_images, kmnist_test_labels))

kmnist_TL.save('/content/drive/MyDrive/saved_models/kmnist_TL.h5', save_format='h5')

/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/15
469/469 [==============================] - 38s 65ms/step - loss: 0.3917 - sparse_categorical_crossentropy: 1.6688 - accuracy: 0.8815 - val_loss: 0.3897 - val_sparse_categorical_crossentropy: 1.6073 - val_accuracy: 0.8843
Epoch 2/15
469/469 [==============================] - 26s 55ms/step - loss: 0.1194 - sparse_categorical_crossentropy: 1.3551 - accuracy: 0.9650 - val_loss: 0.2729 - val_sparse_categorical_crossentropy: 1.4412 - val_accuracy: 0.9226
Epoch 3/15
469/469 [==============================] - 26s 56ms/step - loss: 0.0769 - sparse_categorical_crossentropy: 1.1759 - accuracy: 0.9762 - val_loss: 0.2380 - val_sparse_categorical_crossentropy: 1.2103 - val_accuracy: 0.9358
Epoch 4/15
469/469 [==============================] - 26s 56ms/step - loss: 0.0566 - sparse_categorical_crossentropy: 1.0691 - accuracy: 0.9822 - val_loss: 0.2006 - val_sparse_categorical_crossentropy: 1.1933 - val_accuracy: 0.9452
Epoch 5/15
469/469 [==============================] - 27s 57ms/step - lo

# Creating the Ensemble